In [38]:
import gzip, json, pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn_pandas import DataFrameMapper
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [72]:
input = '../Data/reviews_Movies_and_TV_5.json.gz'
data = []
with gzip.open(input) as f:
    for l in f:
        data.append(json.loads(l.strip()))

data = pd.DataFrame.from_dict(data)
target = data['overall']
text = data['reviewText']

print(sorted(Counter(target).items()))


[(1.0, 104219), (2.0, 102410), (3.0, 201302), (4.0, 382994), (5.0, 906608)]


In [73]:
data.head()
data.drop(['reviewerID', 'asin', 'reviewerName', 'helpful', ])

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,ADZPIG9QOCDG5,0005019281,"Alice L. Larson ""alice-loves-books""","[0, 0]",This is a charming version of the classic Dick...,4.0,good version of a classic,1203984000,"02 26, 2008"
1,A35947ZP82G7JH,0005019281,Amarah Strack,"[0, 0]",It was good but not as emotionally moving as t...,3.0,Good but not as moving,1388361600,"12 30, 2013"
2,A3UORV8A9D5L2E,0005019281,Amazon Customer,"[0, 0]","Don't get me wrong, Winkler is a wonderful cha...",3.0,Winkler's Performance was ok at best!,1388361600,"12 30, 2013"
3,A1VKW06X1O2X7V,0005019281,"Amazon Customer ""Softmill""","[0, 0]",Henry Winkler is very good in this twist on th...,5.0,It's an enjoyable twist on the classic story,1202860800,"02 13, 2008"
4,A3R27T4HADWFFJ,0005019281,BABE,"[0, 0]",This is one of the best Scrooge movies out. H...,4.0,Best Scrooge yet,1387670400,"12 22, 2013"


In [ ]:
# Oversampling the text and target and put them into a new df
# RandomOvesampler und UNdersampler do only work in feature spaces -> manually composing the undersampling
# Unterteilen in Klassen -> von jeder Klasse nur 90000 random
# data.describe()
df_1 = data[data['overall']==1.0].values.tolist()
df_2 = data[data['overall']==2.0].values.tolist()
df_3 = data[data['overall']==3.0].values.tolist()
df_4 = data[data['overall']==4.0].values.tolist()
df_5 = data[data['overall']==5.0].values.tolist()
print(len(df_1),len(df_2), len(df_3), len(df_4), len(df_5))
print(type(df_1))
# data = random.sample(data, 200000)

In [64]:
import random
df1 = random.sample(df_1, 90000)
df2 = random.sample(df_2, 90000)
df3 = random.sample(df_3, 90000)
df4 = random.sample(df_4, 90000)
df5 = random.sample(df_5, 90000)
anh = pd.DataFrame.from_dict(df1)

In [71]:
anh.head()

,0,1,2,3,4,5,6,7,8
0,A2FL0EKQ2EQ9E6,0790743221,"a viewer ""a viewer""","[9, 67]",First let me say this: I AM NOT A KATHARINE HE...,1.0,OVERRATED FILM!!,1091577600,"08 4, 2004"
1,A3HM5G9IV1FTFP,B000FS9UK8,Douglas King,"[3, 7]","I loved the original ""Basic Instinct"". Sure, i...",1.0,What a shame!,1144022400,"04 3, 2006"
2,A4Z9NHOW5LT0M,B000EHQTT0,JoB,"[0, 0]",The movie starts off super slow and basically ...,1.0,Really Bad!,1384819200,"11 19, 2013"
3,A30Q6TIQMXGATR,B000056T5E,pablo,"[2, 4]",here we are to rate this sort of horror film.i...,1.0,horrible!,1002240000,"10 5, 2001"
4,A207IYGZTAF0DS,B0001CVB0U,NoWireHangers,"[1, 8]","From the description of this movie, it sounds ...",1.0,Very bad movie. Not what you expected.,1151193600,"06 25, 2006"


In [ ]:
df = pd.concat([text[:50000], text[:50000]], axis=1)
print(df.shape)
df.columns=['count', 'tfidf']
print(df.describe())
mapper = DataFrameMapper([
    ('count', CountVectorizer()),
    ('tfidf', TfidfVectorizer())
])


In [6]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
count    50000 non-null object
tfidf    50000 non-null object
dtypes: object(2)
memory usage: 98.8 MB


In [ ]:
features = mapper.fit_transform(df)

In [ ]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "
def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    return reviews


In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))


In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))


In [9]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))


SyntaxError: invalid syntax (<ipython-input-9-27cd4a422c43>, line 3)